In [10]:
#Dependencies
from urllib.request import urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import WebDriverException  # Added import
import time
import os
from datetime import datetime, timedelta


In [12]:
#Parameters
url_da = r'https://www.omie.es/es/file-download?parents%5B0%5D=marginalpdbc&filename=marginalpdbc'
url_id = r'https://www.omie.es/es/file-download?parents%5B0%5D=precios_pibcic&filename=precios_pibcic'
download_dir = r"C:\Users\Downloads"

exceptions = []
years = [2021, 2022]

In [13]:
#Chrome driver setup
chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
    "profile.default_content_settings.popups": 0,
    "profile.default_content_setting_values.automatic_downloads": 1
}

headless = False
if headless:
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")                                                                                                                                       
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)



NoSuchDriverException: Message: Unable to obtain driver for chrome; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors/driver_location


In [ ]:
#Energy markets data
try:
    for file_type, base_url in [('DA', url_da), ('ID', url_id)]:
        for year in years:
            start_date = datetime(year, 1, 1)
            end_date = datetime(year + 1, 1, 1)
            days = (end_date - start_date).days
            for i in range(days):
                start_date = datetime(2023, 1, 1)
                date = (start_date + timedelta(days=i)).strftime('%Y%m%d')
                file_url = f"{base_url}_{date}.1"
                before_download = set(os.listdir(download_dir))
                try:
                    driver.get(file_url)
                except WebDriverException as e:
                    print(f"Error navigating to {file_url}: {e}")
                    exceptions.append(file_url)
                    continue
                download_complete = False
                for _ in range(5):
                    time.sleep(6)
                    after_download = set(os.listdir(download_dir))
                    new_files = after_download - before_download
                    if new_files:
                        download_complete = True
                        downloaded_file = new_files.pop()
                        break
                
                if not download_complete:
                    print(f"Download for {file_url} timed out.")
                    continue

                old_path = os.path.join(download_dir, downloaded_file)
                new_path = os.path.join(download_dir, f"{downloaded_file}.csv")
                os.rename(old_path, new_path)
                print(f"Downloaded and renamed file to: {new_path}")
finally:
    driver.quit()

print("Download process completed.")
exceptions and print("The following URLs failed to download:")

In [ ]:
#Gas prices data
try:
    for y in years:
        base_url = "https://www.mibgas.es/en/file-access/MIBGAS_Data_" + str(y) + ".csv?path=AGNO_" + str(y) + "/XLS"
        before_download = set(os.listdir(download_dir))
        try:
            driver.get(base_url)
        except WebDriverException as e:
            print(f"Error navigating to {base_url}: {e}")
            exceptions.append(base_url)
            continue
        download_complete = False
        for _ in range(5):
            time.sleep(6)
            after_download = set(os.listdir(download_dir))
            new_files = after_download - before_download
            if new_files:
                download_complete = True
                downloaded_file = new_files.pop()
                break
        
        if not download_complete:
            print(f"Download for {base_url} timed out.")
            continue
        print(f"Downloaded file to: {download_dir}")
finally:
    driver.quit()

In [16]:
import os
import requests
from datetime import datetime, timedelta

# Parameters
url_da = r'https://www.omie.es/es/file-download?parents%5B0%5D=marginalpdbc&filename=marginalpdbc'
url_id = r'https://www.omie.es/es/file-download?parents%5B0%5D=precios_pibcic&filename=precios_pibcic'
download_dir = r"C:\Users\Downloads"

years = [2021, 2022]
exceptions = []

# Ensure the download directory exists
os.makedirs(download_dir, exist_ok=True)

# Function to download a file
def download_file(file_url, save_path):
    try:
        headers = {"User-Agent": "Mozilla/5.0"}
        response = requests.get(file_url, headers=headers, stream=True)
        response.raise_for_status()

        # Debug response content
        print(f"Response for {file_url}: {response.status_code}, Content-Length: {len(response.content)}")

        with open(save_path, 'wb') as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Downloaded: {save_path}")
    except Exception as e:
        print(f"Failed to download {file_url}: {e}")
        exceptions.append(file_url)


# Loop through each year and date
for file_type, base_url in [('DA', url_da), ('ID', url_id)]:
    for year in years:
        start_date = datetime(year, 1, 1)
        end_date = datetime(year + 1, 1, 1)
        days = (end_date - start_date).days
        for i in range(days):
            date = (start_date + timedelta(days=i)).strftime('%Y%m%d')
            file_url = f"{base_url}_{date}.1"
            save_path = os.path.join(download_dir, f"{file_type}_{date}.csv")
            
            # Check if the file already exists
            if os.path.exists(save_path):
                print(f"File already exists: {save_path}")
                continue
            
            # Download the file
            download_file(file_url, save_path)

print("Download process completed.")
if exceptions:
    print("The following URLs failed to download:")
    for url in exceptions:
        print(url)


File already exists: C:\Users\Downloads/DA_20210101.csv
File already exists: C:\Users\Downloads/DA_20210102.csv
File already exists: C:\Users\Downloads/DA_20210103.csv
File already exists: C:\Users\Downloads/DA_20210104.csv
File already exists: C:\Users\Downloads/DA_20210105.csv
File already exists: C:\Users\Downloads/DA_20210106.csv
File already exists: C:\Users\Downloads/DA_20210107.csv
File already exists: C:\Users\Downloads/DA_20210108.csv
File already exists: C:\Users\Downloads/DA_20210109.csv
File already exists: C:\Users\Downloads/DA_20210110.csv
File already exists: C:\Users\Downloads/DA_20210111.csv
File already exists: C:\Users\Downloads/DA_20210112.csv
File already exists: C:\Users\Downloads/DA_20210113.csv
File already exists: C:\Users\Downloads/DA_20210114.csv
File already exists: C:\Users\Downloads/DA_20210115.csv
File already exists: C:\Users\Downloads/DA_20210116.csv
File already exists: C:\Users\Downloads/DA_20210117.csv
File already exists: C:\Users\Downloads/DA_20210

KeyboardInterrupt: 

In [3]:
import os
import time
from selenium import webdriver
from selenium.common.exceptions import WebDriverException

# Define years and download directory
years = range(2018, 2023)  # Adjust year range as needed
download_dir = "/home/teitur/DTU/electricproject/deeplearning/gas"  # Update this with the correct path
exceptions = []

# Initialize WebDriver (configure as needed for your browser)
driver = webdriver.Chrome()  # Use the appropriate WebDriver for your browser

try:
    for y in years:
        base_url = f"https://www.mibgas.es/en/file-access/MIBGAS_Data_{y}.xlsx"
        print(f"Attempting to download: {base_url}")
        before_download = set(os.listdir(download_dir))
        
        try:
            driver.get(base_url)
        except WebDriverException as e:
            print(f"Error navigating to {base_url}: {e}")
            exceptions.append(base_url)
            continue
        
        # Wait for the download to complete
        download_complete = False
        for _ in range(5):  # Retry up to 5 times
            time.sleep(6)  # Check every 6 seconds
            after_download = set(os.listdir(download_dir))
            new_files = after_download - before_download
            if new_files:
                download_complete = True
                downloaded_file = new_files.pop()
                
                # Rename the file to include the year
                new_file_path = os.path.join(download_dir, f"MIBGAS_Data_{y}.csv")
                os.rename(os.path.join(download_dir, downloaded_file), new_file_path)
                print(f"Downloaded and renamed file to: {new_file_path}")
                break

        if not download_complete:
            print(f"Download for {base_url} timed out.")
            exceptions.append(base_url)
            continue
finally:
    # Ensure the driver is properly closed
    driver.quit()

# Print summary of any issues
if exceptions:
    print("The following downloads failed:")
    for url in exceptions:
        print(url)
else:
    print("All downloads completed successfully!")


Attempting to download: https://www.mibgas.es/en/file-access/MIBGAS_Data_2018.xlsx
Download for https://www.mibgas.es/en/file-access/MIBGAS_Data_2018.xlsx timed out.
Attempting to download: https://www.mibgas.es/en/file-access/MIBGAS_Data_2019.xlsx
Error navigating to https://www.mibgas.es/en/file-access/MIBGAS_Data_2019.xlsx: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.69)
Stacktrace:
#0 0x55e81e9b94ca <unknown>
#1 0x55e81e4cc620 <unknown>
#2 0x55e81e4a22ef <unknown>
#3 0x55e81e5477ad <unknown>
#4 0x55e81e55d79c <unknown>
#5 0x55e81e53f113 <unknown>
#6 0x55e81e50dbe0 <unknown>
#7 0x55e81e50ebbe <unknown>
#8 0x55e81e985e4b <unknown>
#9 0x55e81e989de2 <unknown>
#10 0x55e81e972d2c <unknown>
#11 0x55e81e98a957 <unknown>
#12 0x55e81e9584bf <unknown>
#13 0x55e81e9a8348 <unknown>
#14 0x55e81e9a8510 <unknown>
#15 0x55e81e9b8346 <unknown>
#16 0x7f28c0705ac3 <unknown>

Attempting to download: https://www.mibgas.